In [1]:
# coding: utf-8
import re
import math
import csv
import sys
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from obspy import UTCDateTime, Stream, Trace

%matplotlib inline

In [2]:
def unpackAfile(infile):

# == opening Afile ==
    b = os.path.getsize(infile)
    FH = open(infile, 'rb')
    line = FH.read(b)
    fileHeader = struct.unpack("<4s3h6bh6s", line[0:24])

    fileLength = fileHeader[3]
    port = fileHeader[10]
    # FirstStn = fileHeader[11][0:4].decode('ASCII').rstrip()
# =================================Header=================================

    portHeader = []
    for i in range(24, port * 32, 32):
        port_data = struct.unpack("<4s4s3sbh2b4s12b", line[i:i+32])
        portHeader.append(port_data)

# =================================Data===================================

    dataStartByte = 24+int(port)*32
    dataPoint = 3*int(port)*int(fileLength)*100
    times = int(port)*3*4
    data = []

    data = struct.unpack("<%di" % dataPoint, line[dataStartByte:dataStartByte + dataPoint*4])


    portHeader = np.array(portHeader)
    data = np.array(data)
    idata = data.reshape((3,port,fileLength*100),order='F')

#== write to obspy Stream --
    sttime = UTCDateTime(fileHeader[1], fileHeader[4], fileHeader[5], fileHeader[6], fileHeader[7], fileHeader[8], fileHeader[2])
    npts = fileHeader[3]*fileHeader[9]
    samp = fileHeader[9]
    afst = Stream()
    
    for stc in range(fileHeader[10]):
        stn = portHeader[stc][0].decode('ASCII').rstrip()
        instrument = portHeader[stc][1].decode('ASCII').rstrip()
        loc = '0'+str(portHeader[stc][6].decode('ASCII'))
        net = str(portHeader[stc][7].decode('ASCII')).rstrip()
        GPS = int(portHeader[stc][3])
        
        # remove GPS unlock or broken station
        if ( 2>1):
            chc = 0
            if instrument == 'FBA':
                chc = 1
            elif instrument == 'SP':
                chc = 4
            elif instrument == 'BB':
                chc = 7
            
            for ch in range(3):
                chn = 'Ch'+str(chc+ch)
                
                stats = {'network': net, 'station': stn, 'location': loc,
                        'channel': chn, 'npts': npts, 'sampling_rate': samp,
                        'starttime': sttime}
                
                data = np.array(idata[ch][stc], dtype=float)
                sttmp = Stream([Trace(data=data, header=stats)])
                afst += sttmp

    return afst

In [3]:
# Read CWB Waveform file
st=unpackAfile("02080054.18C")

In [4]:
# select Network "SMT"
select_wav = st.select(network="SMT",channel="Ch1")

In [5]:
select_wav[0].stats.starttime

2018-02-08T00:54:00.000000Z

In [6]:
yr=select_wav[0].stats.starttime.year
mo=select_wav[0].stats.starttime.month
dy=select_wav[0].stats.starttime.day
hr=select_wav[0].stats.starttime.hour
mn=select_wav[0].stats.starttime.minute
se=select_wav[0].stats.starttime.second

In [7]:
f"{yr}{mo}{dy}{hr}{mn}{se}"  

'2018280540'

In [8]:
f"{yr}{mo:02d}{dy:02d}{hr:02d}{mn:02d}{se:02d}"  

'20180208005400'

In [9]:
f"{yr}-{mo:02d}-{dy:02d}-{hr:02d}-{mn:02d}-{se:02d}"  

'2018-02-08-00-54-00'

In [10]:
select_staname = [ i.stats.station for i in select_wav ]


In [11]:
with open(r"D:\python\0602\PhaseNet-master\mydata\fname.csv", 'w') as fp:
    fp.write("fname\n")
    for ii in select_staname: 
        for i in range(2):
            Z=st.select(station=ii,channel="Ch1")[0].data[3000*i:3000*(i+1)]*1.0
            N=st.select(station=ii,channel="Ch2")[0].data[3000*i:3000*(i+1)]*1.0
            E=st.select(station=ii,channel="Ch3")[0].data[3000*i:3000*(i+1)]*1.0
            waveform=np.vstack((Z,N,E)).T
            
            time = f"{yr}-{mo:02d}-{dy:02d}-{hr:02d}-{mn:02d}-{se:02d}"  
            
            ffname = f"D:\\python\\0602\\PhaseNet-master\\mydata\\{ii}.{ii}.{time}.{i}.npz"
            np.savez(ffname, data=waveform)
            print(ffname)
            ffname = f"{ffname}\n"
            fp.write(ffname)

D:\python\0602\PhaseNet-master\mydata\ALS.ALS.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\ALS.ALS.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\ANP.ANP.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\ANP.ANP.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\BAC.BAC.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\BAC.BAC.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\CHK.CHK.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\CHK.CHK.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\CHN1.CHN1.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\CHN1.CHN1.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\CHN2.CHN2.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\CHN2.CHN2.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\CHN3.CHN3.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\CHN3.CHN3.2018-0

D:\python\0602\PhaseNet-master\mydata\SSH.SSH.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\STY.STY.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\STY.STY.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\TAI.TAI.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\TAI.TAI.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\TAI1.TAI1.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\TAI1.TAI1.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\TAP.TAP.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\TAP.TAP.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\TAW.TAW.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\TAW.TAW.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\TCU.TCU.2018-02-08-00-54-00.0.npz
D:\python\0602\PhaseNet-master\mydata\TCU.TCU.2018-02-08-00-54-00.1.npz
D:\python\0602\PhaseNet-master\mydata\TTN.TTN.2018-02-08-00-